In [1]:
# Imports
import numpy as np
from QSSBuilder import QSSBuilder

# Calculating steady states.

In [2]:
# Build DataFrames.
# qss = QSSBuilder("MegAWES_SixDofPaper")  # Only first time.
qss = QSSBuilder.from_csv("MegAWES_SixDofPaper")

Done loading MegAWES_SixDofPaper.csv


In [3]:
qss.add_steady_states(
    {
        "vw_mps": np.unique(np.linspace(0, 30, 13, dtype=int)),
        "Lt_m": np.unique(np.linspace(500, 1500, 3, dtype=int)),
        "phi_deg": np.unique(np.linspace(0, 30, 3, dtype=int)),
        "beta_deg": np.unique(np.linspace(0, 45, 4, dtype=int)),
        "chi_deg": np.unique(np.linspace(0, 180, 5, dtype=int)),
        # TODO: Smarter way to find different Ftk_N that work than looping over all values.
        # Use dtype int for the merge into an existing DataFrame to be less error-prone.
        # Otherwise it sometimes couldn't match two floats with each other. Integer
        # precision is also more than enough here.
        # "Ftk_N": np.unique(np.logspace(1, 7, 103, dtype=int)),
        "Ftk_N": np.unique(np.arange(10, 1e7, 10000, dtype=int)),
    }
)
qss.save_df()

Done making new DataFrame with all possible combinations in ranges.
Merged 1872000 new rows into                   the DataFrame.


Calculating quasi-steady states:  65%|██████▌   | 1531178/2340000 [13:06<10:00, 1347.54it/s] 

In [ ]:
# qss_massless = QSSBuilder("MegAWES_massless")  # Only first time.
qss_massless = QSSBuilder.from_csv("MegAWES_massless")

Done loading MegAWES_massless.csv


In [ ]:
qss_massless.add_steady_states(
    {
        "vw_mps": np.unique(np.linspace(0, 30, 13, dtype=int)),
        "Lt_m": np.unique(np.linspace(500, 1500, 3, dtype=int)),
        "phi_deg": np.unique(np.linspace(0, 30, 3, dtype=int)),
        "beta_deg": np.unique(np.linspace(0, 45, 4, dtype=int)),
        # Massless doesn't need to very chi_deg because it has no effect.
        "chi_deg": [90],
        # Use dtype int for the merge into an existing DataFrame to be less error-prone.
        # Otherwise it sometimes couldn't match two floats with each other. Integer
        # precision is also more than enough here.
        # "Ftk_N": np.unique(np.logspace(1, 7, 103, dtype=int)),
        "Ftk_N": np.unique(np.arange(10, 1e7, 10000, dtype=int)),
    }
)
qss_massless.save_df()

Done making new DataFrame with all possible combinations in ranges.
Merged 0 new rows into                   the DataFrame.


Calculating quasi-steady states: 100%|██████████| 93600/93600 [00:00<00:00, 507641.85it/s]


Saving DataFrame of quasi-steady state builder for MegAWES_massless as MegAWES_massless.csv


# Cleaning the data

I saw that there were some anomalies when plotting the Power or tether force over reeling factor. Clearly there were some outliers. By sorting each state by 'f' and then seeing where there is a larger than normal difference in Ftg_N, these datapoints can be taken out.

# !!!! This method only works once !!!! it doesn't work if you append to the DataFrame!
# Better to do Ftg_N.diff / f.diff() and take out values that don't fall smoothly on that line.

In [ ]:
# TODO: make this also into a python class and save as a separate csv.
# TODO: Needed? when looking at df_star I think it's actually fine already.

In [6]:
# Imports
import pandas as pd
# import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [195]:
# Check if data looks ok.
df = qss.df.loc[
    (qss.df['beta_deg'] == 0)
    & (qss.df['phi_deg'] == 0)
    & (qss.df['chi_deg'] == 180)
    & (qss.df['Lt_m'] == 1500)
    & (qss.df['vw_mps'] == 10)
    # & (qss.df['P_W'] > -1e6)
].sort_values(by='vr_mps')

fig = px.line(df, x='vr_mps', y='Ftg_N', color='vw_mps', markers=True, symbol='chi_deg')
fig.show()

In [200]:

df['d_Ftg_N_df'] = df['Ftg_N'].diff() / df['f'].diff()
df['d_Ftg_N_df_diff'] = df['d_Ftg_N_df'].diff().diff().diff().diff()
fig = px.line(df, x='f', y='d_Ftg_N_df_diff', color='vw_mps', markers=True, symbol='beta_deg')
fig.show()

In [67]:
df.drop(df.loc[np.isclose(df['Ftg_N_diff'], -50000, atol=1)].index)

,vw_mps,Lt_m,phi_deg,beta_deg,chi_deg,Ftk_N,calculated_SS,f,Ftg_N,P_W,vr_mps,Ftg_N_diff
237599,15,1000,30,15,90,9950010,True,-1.111885,9.949583e+06,-1.659419e+08,-16.678281,NaN
237401,15,1000,30,15,90,50010,True,0.352058,4.958297e+04,2.618409e+05,5.280864,-499999.945748
237410,15,1000,30,15,90,500010,True,0.358905,4.995829e+05,2.689543e+06,5.383577,449999.945842
237402,15,1000,30,15,90,100010,True,0.450830,9.958293e+04,6.734248e+05,6.762453,-149999.988555
237404,15,1000,30,15,90,200010,True,0.461211,1.995829e+05,1.380747e+06,6.918161,99999.989779
237400,15,1000,30,15,90,10,True,NaN,NaN,NaN,NaN,NaN


In [72]:
qss_temp.df.shape

(468000, 11)

In [80]:
indices_to_delete = []

for beta_deg in qss_temp.df['beta_deg'].unique():
    for phi_deg in qss_temp.df['phi_deg'].unique():
        for chi_deg in qss_temp.df['chi_deg'].unique():
            for Lt_m in qss_temp.df['Lt_m'].unique():
                for vw_mps in qss_temp.df['vw_mps'].unique():
                    df = qss_temp.df[
                        (qss_temp.df['beta_deg'] == beta_deg)
                        & (qss_temp.df['phi_deg'] == phi_deg)
                        & (qss_temp.df['chi_deg'] == chi_deg)
                        & (qss_temp.df['Lt_m'] == Lt_m)
                        & (qss_temp.df['vw_mps'] == vw_mps)
                    ].sort_values(by='f')

                    df['Ftg_N_diff'] = df['Ftg_N'].diff()
                    indices_to_delete.append(df.loc[~np.isclose(df['Ftg_N_diff'], -50000, atol=1)].index)

In [81]:
indices_to_delete

[Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
             ...
             190, 191, 192, 193, 194, 195, 196, 197, 198, 199],
            dtype='int64', length=200),
 Int64Index([36199, 36000, 36001, 36002, 36003, 36004, 36005, 36006, 36007,
             36008, 36009, 36010, 36011, 36012],
            dtype='int64'),
 Int64Index([72199, 72000, 72001, 72002, 72003, 72004, 72005, 72006, 72007,
             72008, 72009, 72010, 72011, 72012],
            dtype='int64'),
 Int64Index([108199, 108000, 108001, 108002, 108003, 108004, 108005, 108006,
             108007, 108008, 108009, 108010, 108011, 108012],
            dtype='int64'),
 Int64Index([144199, 144000, 144001, 144002, 144003, 144004, 144005, 144006,
             144007, 144008, 144009, 144010, 144011, 144012],
            dtype='int64'),
 Int64Index([180199, 180000, 180001, 180002, 180003, 180004, 180005, 180006,
             180007, 180008, 180009, 180010, 180011, 180012],
            dtype='int64'),
 Int64Ind

In [82]:
for idx in indices_to_delete:
    qss_temp.df.drop(idx, inplace=True)

In [100]:
qss.df.shape

(409579, 11)

In [99]:
# Check if that helped.
df = qss_temp.df.loc[
    # (qss_temp.df['beta_deg'] == 15)
    (qss_temp.df['phi_deg'] == 0)
    & (qss_temp.df['chi_deg'] == 135)
    & (qss_temp.df['Lt_m'] == 1500)
    # & (qss_temp.df['vw_mps'] == 15)
    # & (qss.df['P_W'] > -1e6)
].sort_values(by='f')

fig = px.line(df, x='f', y='Ftg_N', color='vw_mps', markers=True, symbol='beta_deg')
fig.show()

# Processing

In [113]:
# Finding the best power for all states.
temp = qss_temp.df.sort_values('P_W', ascending=False)
qss_star = temp.drop_duplicates(['beta_deg', 'phi_deg', 'chi_deg', 'Lt_m', 'vw_mps'], keep='first')

# Plotting

In [184]:
df = qss_star.loc[
    (qss_star['beta_deg'] == 0)
    & (qss_star['phi_deg'] == 0)
    # & (qss_star['chi_deg'] == 135)
    & (qss_star['Lt_m'] == 1500)
    # & (qss_star['vw_mps'] == 15)
].sort_values(by='vr_mps')

fig = px.line(df, x='vr_mps', y='Ftg_N', markers=True, color='chi_deg')
fig.show()

In [183]:
fig = go.Figure()

chi_deg = 0

for vw_mps in np.unique(qss.df['vw_mps']):
    # Check if data looks ok.
    df = qss.df.loc[
        (qss.df['beta_deg'] == 0)
        & (qss.df['phi_deg'] == 0)
        & (qss.df['chi_deg'] == chi_deg)
        & (qss.df['Lt_m'] == 1500)
        & (qss.df['vw_mps'] == vw_mps)
        # & (qss.df['P_W'] > -1e6)
    ].sort_values(by='vr_mps')

    fig.add_trace(go.Scatter(x=df['vr_mps'], y=df['Ftg_N'], name=f"{vw_mps=}"))

df = qss_star.loc[
    (qss_star['beta_deg'] == 0)
    & (qss_star['phi_deg'] == 0)
    & (qss_star['chi_deg'] == chi_deg)
    & (qss_star['Lt_m'] == 1500)
    # & (qss_star['vw_mps'] == 15)
].sort_values(by='vr_mps')
fig.add_trace(go.Scatter(x=df['vr_mps'], y=df['Ftg_N'], name='optimum'))
fig.show()